# Importing Python modules

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['text.usetex']= True
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Importing self-written functions

In [4]:
from pathlib import Path
import sys

PROJECT_ROOT = Path().resolve().parent
print(f'root directory: {PROJECT_ROOT}')
sys.path.insert(0, str(PROJECT_ROOT / "packages"))  # moving directory to 'packages' folder to import functions
print(f'current system path: {sys.path[0]}')

from conversions_calculations_and_constants.conversions import M_ab_to_L_nu
from conversions_calculations_and_constants.dust_correction import extinction_k


root directory: /Users/MarissaPerry/Documents/undergrad/projects/capstone
current system path: /Users/MarissaPerry/Documents/undergrad/projects/capstone/packages


# Loading in sample

In [5]:
NIRSpec_sample_df = pd.read_csv('../outputs/checkpoints/sample_Balmer.csv')
NIRSpec_sample_df

,MSA_ID,SURVEY,SPECTRA TYPE,Z_SPEC,RA,DEC,H-ALPHA INTEGRATED LUMINOSITY,H-ALPHA INTEGRATED LUMINOSITY MEAN ERROR,H-ALPHA INTEGRATED LUMINOSITY 16% ERROR,H-ALPHA INTEGRATED LUMINOSITY 84% ERROR,...,MASS 16% ERROR,MASS MEDIAN,MASS 84% ERROR,ABSOLUTE MAG 1500 84% ERROR,ABSOLUTE MAG 1500 MEDIAN,ABSOLUTE MAG 1500 16% ERROR,REDDENING,REDDENING ERR,A_V,A_V ERR
0,323,CEERS,GRATING,5.6670,215.125247,53.004816,2.890517e+42,5.500441e+41,6.499634e+41,5.147870e+41,...,9.997739,10.137217,10.643369,-22.583353,-22.502931,-22.477902,0.000000,0.000000,0.000000,0.000000
1,355,CEERS,GRATING,6.1000,215.151954,52.992771,2.524020e+42,4.020752e+41,4.026490e+41,4.015207e+41,...,10.790051,10.853662,10.927516,-23.660963,-23.616570,-23.582830,0.091926,0.110822,0.372301,0.448831
2,403,CEERS,GRATING,5.7610,214.885556,52.835043,4.193009e+42,7.291792e+41,7.269488e+41,7.314849e+41,...,10.620024,10.699345,10.754652,-23.293149,-23.269745,-23.247962,0.303005,0.123527,1.227172,0.500283
3,603,CEERS,GRATING,6.0580,215.032515,52.897511,2.323581e+42,4.094067e+41,4.092556e+41,4.096007e+41,...,9.455681,9.511468,9.560115,-20.801093,-20.756414,-20.716712,0.052216,0.124113,0.211476,0.502659
4,618,CEERS,GRATING,6.0510,215.126857,52.968083,1.447606e+42,6.203280e+41,6.227582e+41,6.181015e+41,...,8.111941,8.342621,8.618631,-20.079706,-19.965951,-19.817897,0.357337,0.311711,1.447215,1.262428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,959801,RUBIES,PRISM AND GRATING,6.7292,214.800187,52.814506,7.300656e+41,7.639598e+40,7.686104e+40,7.594264e+40,...,8.235452,8.303398,8.373227,-20.286263,-20.235605,-20.156063,0.000000,0.000000,0.000000,0.000000
341,966377,RUBIES,PRISM,6.6784,214.978311,52.953821,3.021473e+41,6.360257e+40,6.115780e+40,6.606596e+40,...,8.444550,8.517332,8.575585,-20.544037,-20.483902,-20.424300,0.000000,0.000000,0.000000,0.000000
342,970128,RUBIES,GRATING,6.4858,214.880997,52.891214,1.100483e+42,8.597488e+40,8.403987e+40,8.793203e+40,...,8.296121,8.343320,8.398535,-20.408952,-20.367815,-20.325321,0.000000,0.000000,0.000000,0.000000
343,975193,RUBIES,PRISM,6.9901,214.967248,52.963630,1.031260e+42,8.545624e+40,8.388811e+40,8.702756e+40,...,8.028886,8.093625,8.190734,-19.729947,-19.672063,-19.606486,0.000000,0.000000,0.000000,0.000000


# Bagpipes SED-fitting results

## Convert AB magnitude to luminosity and dust correct

In [6]:
# constants --------------------------
wav_rest_angstrom = 1500                 # [A]
wav_microns = wav_rest_angstrom * 1e-4  # [A] to [µm]
c = 2.99792458e18                       # [A/s]
nu_1500 = c / wav_rest_angstrom         # [Hz]
k_lambda = extinction_k(wav_microns)
attenuation_factor = 0.44               # differential attenuation: stellar reddening = 0.44 x nebular reddening
# ------------------------------------

df = NIRSpec_sample_df.copy()

for measurement_type in ['16% ERROR', 'MEDIAN', '84% ERROR']:
    mag_col = f'ABSOLUTE MAG 1500 {measurement_type}'
    lnu_corr_col = f'UV LUMINOSITY DENSITY DUST-CORRECTED {measurement_type}'
    l_corr_col = f'UV LUMINOSITY DUST-CORRECTED {measurement_type}'

    # 1. Convert M_AB → L_nu
    L_nu = M_ab_to_L_nu(df[mag_col])

    # 2. Apply dust correction
    E_BV_nebular = df['REDDENING']
    E_BV_stellar = attenuation_factor * E_BV_nebular
    power_term = 0.4 * E_BV_stellar * k_lambda  # from Calzetti 2001 equation (7)
    L_nu_corr = L_nu*(10**power_term)              # ''

    # 3. Convert L_nu → L (erg/s)
    L_corr = L_nu_corr * nu_1500

    df[lnu_corr_col] = L_nu_corr
    df[l_corr_col] = L_corr

df

,MSA_ID,SURVEY,SPECTRA TYPE,Z_SPEC,RA,DEC,H-ALPHA INTEGRATED LUMINOSITY,H-ALPHA INTEGRATED LUMINOSITY MEAN ERROR,H-ALPHA INTEGRATED LUMINOSITY 16% ERROR,H-ALPHA INTEGRATED LUMINOSITY 84% ERROR,...,REDDENING,REDDENING ERR,A_V,A_V ERR,UV LUMINOSITY DENSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DENSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DUST-CORRECTED 84% ERROR
0,323,CEERS,GRATING,5.6670,215.125247,53.004816,2.890517e+42,5.500441e+41,6.499634e+41,5.147870e+41,...,0.000000,0.000000,0.000000,0.000000,4.257835e+29,8.509780e+44,4.357127e+29,8.708226e+44,4.692123e+29,9.377753e+44
1,355,CEERS,GRATING,6.1000,215.151954,52.992771,2.524020e+42,4.020752e+41,4.026490e+41,4.015207e+41,...,0.091926,0.110822,0.372301,0.448831,1.395383e+30,2.788836e+45,1.439428e+30,2.876864e+45,1.499501e+30,2.996928e+45
2,403,CEERS,GRATING,5.7610,214.885556,52.835043,4.193009e+42,7.291792e+41,7.269488e+41,7.314849e+41,...,0.303005,0.123527,1.227172,0.500283,1.512146e+30,3.022199e+45,1.542789e+30,3.083444e+45,1.576406e+30,3.150632e+45
3,603,CEERS,GRATING,6.0580,215.032515,52.897511,2.323581e+42,4.094067e+41,4.092556e+41,4.096007e+41,...,0.052216,0.124113,0.211476,0.502659,9.257243e+28,1.850168e+44,9.602021e+28,1.919076e+44,1.000540e+29,1.999695e+44
4,618,CEERS,GRATING,6.0510,215.126857,52.968083,1.447606e+42,6.203280e+41,6.227582e+41,6.181015e+41,...,0.357337,0.311711,1.447215,1.262428,7.096244e+28,1.418267e+44,8.132993e+28,1.625473e+44,9.031340e+28,1.805018e+44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,959801,RUBIES,PRISM AND GRATING,6.7292,214.800187,52.814506,7.300656e+41,7.639598e+40,7.686104e+40,7.594264e+40,...,0.000000,0.000000,0.000000,0.000000,5.017103e+28,1.002727e+44,5.398459e+28,1.078945e+44,5.656307e+28,1.130479e+44
341,966377,RUBIES,PRISM,6.6784,214.978311,52.953821,3.021473e+41,6.360257e+40,6.115780e+40,6.606596e+40,...,0.000000,0.000000,0.000000,0.000000,6.423147e+28,1.283741e+44,6.785605e+28,1.356182e+44,7.172044e+28,1.433416e+44
342,970128,RUBIES,GRATING,6.4858,214.880997,52.891214,1.100483e+42,8.597488e+40,8.403987e+40,8.793203e+40,...,0.000000,0.000000,0.000000,0.000000,5.863490e+28,1.171887e+44,6.097533e+28,1.218663e+44,6.332991e+28,1.265722e+44
343,975193,RUBIES,PRISM,6.9901,214.967248,52.963630,1.031260e+42,8.545624e+40,8.388811e+40,8.702756e+40,...,0.000000,0.000000,0.000000,0.000000,3.024284e+28,6.044383e+43,3.212576e+28,6.420706e+43,3.388494e+28,6.772300e+43


In [7]:
df['UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN'].describe()

count    3.450000e+02
mean     8.352971e+28
std      1.632093e+29
min      5.320656e+25
25%      1.899046e+28
50%      3.963561e+28
75%      7.771412e+28
max      1.542789e+30
Name: UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN, dtype: float64

# Signal-to-noise cut

In [8]:
# computing mean error for both UV luminosities -----------------
lum_col = 'UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN'
err_low_col = 'UV LUMINOSITY DENSITY DUST-CORRECTED 16% ERROR'
err_high_col = 'UV LUMINOSITY DENSITY DUST-CORRECTED 84% ERROR'
df['UV LUMINOSITY DENSITY DUST-CORRECTED MEAN ERROR'] = 0.5 * (df[err_high_col] - df[err_low_col])  # symmetric error

lum_col = 'UV LUMINOSITY DUST-CORRECTED MEDIAN'
err_low_col = 'UV LUMINOSITY DUST-CORRECTED 16% ERROR'
err_high_col = 'UV LUMINOSITY DUST-CORRECTED 84% ERROR'
df['UV LUMINOSITY DUST-CORRECTED MEAN ERROR'] = 0.5 * (df[err_high_col] - df[err_low_col])  # symmetric error
# ---------------------------------------------------------------

df['UV LUMINOSITY SNR'] = df[lum_col] / df['UV LUMINOSITY DUST-CORRECTED MEAN ERROR']

# SNR < 3 cut
cut_sources = df[df['UV LUMINOSITY SNR'] < 3]
print('sources with UV continuum luminosity SNR < 3:')
cut_sources

sources with UV continuum luminosity SNR < 3:


,MSA_ID,SURVEY,SPECTRA TYPE,Z_SPEC,RA,DEC,H-ALPHA INTEGRATED LUMINOSITY,H-ALPHA INTEGRATED LUMINOSITY MEAN ERROR,H-ALPHA INTEGRATED LUMINOSITY 16% ERROR,H-ALPHA INTEGRATED LUMINOSITY 84% ERROR,...,A_V ERR,UV LUMINOSITY DENSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DENSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEAN ERROR,UV LUMINOSITY DUST-CORRECTED MEAN ERROR,UV LUMINOSITY SNR
71,12286,CEERS,GRATING,2.9550,215.154208,52.955847,7.570337e+41,2.854931e+41,2.853812e+41,2.856072e+41,...,1.118974,1.310792e+27,2.619771e+42,2.194948e+27,4.386860e+42,3.105913e+27,6.207529e+42,8.975605e+26,1.793879e+42,2.445460
100,11917,RUBIES,PRISM,2.5909,34.268266,-5.295208,2.731768e+41,2.702650e+40,2.665470e+40,2.741120e+40,...,0.000000,3.143229e+24,6.282110e+39,5.320656e+25,1.063395e+41,4.555624e+26,9.104945e+41,2.262096e+26,4.521062e+41,0.235209


In [9]:
good_snr_sources = df[df['UV LUMINOSITY SNR'] >= 3]
good_snr_sources.reset_index(drop=True, inplace=True)
good_snr_sources

,MSA_ID,SURVEY,SPECTRA TYPE,Z_SPEC,RA,DEC,H-ALPHA INTEGRATED LUMINOSITY,H-ALPHA INTEGRATED LUMINOSITY MEAN ERROR,H-ALPHA INTEGRATED LUMINOSITY 16% ERROR,H-ALPHA INTEGRATED LUMINOSITY 84% ERROR,...,A_V ERR,UV LUMINOSITY DENSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DENSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEAN ERROR,UV LUMINOSITY DUST-CORRECTED MEAN ERROR,UV LUMINOSITY SNR
0,323,CEERS,GRATING,5.6670,215.125247,53.004816,2.890517e+42,5.500441e+41,6.499634e+41,5.147870e+41,...,0.000000,4.257835e+29,8.509780e+44,4.357127e+29,8.708226e+44,4.692123e+29,9.377753e+44,2.171436e+28,4.339867e+43,20.065652
1,355,CEERS,GRATING,6.1000,215.151954,52.992771,2.524020e+42,4.020752e+41,4.026490e+41,4.015207e+41,...,0.448831,1.395383e+30,2.788836e+45,1.439428e+30,2.876864e+45,1.499501e+30,2.996928e+45,5.205892e+28,1.040458e+44,27.649978
2,403,CEERS,GRATING,5.7610,214.885556,52.835043,4.193009e+42,7.291792e+41,7.269488e+41,7.314849e+41,...,0.500283,1.512146e+30,3.022199e+45,1.542789e+30,3.083444e+45,1.576406e+30,3.150632e+45,3.213034e+28,6.421622e+43,48.016593
3,603,CEERS,GRATING,6.0580,215.032515,52.897511,2.323581e+42,4.094067e+41,4.092556e+41,4.096007e+41,...,0.502659,9.257243e+28,1.850168e+44,9.602021e+28,1.919076e+44,1.000540e+29,1.999695e+44,3.740775e+27,7.476374e+42,25.668534
4,618,CEERS,GRATING,6.0510,215.126857,52.968083,1.447606e+42,6.203280e+41,6.227582e+41,6.181015e+41,...,1.262428,7.096244e+28,1.418267e+44,8.132993e+28,1.625473e+44,9.031340e+28,1.805018e+44,9.675484e+27,1.933758e+43,8.405774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,959801,RUBIES,PRISM AND GRATING,6.7292,214.800187,52.814506,7.300656e+41,7.639598e+40,7.686104e+40,7.594264e+40,...,0.000000,5.017103e+28,1.002727e+44,5.398459e+28,1.078945e+44,5.656307e+28,1.130479e+44,3.196019e+27,6.387616e+42,16.891200
339,966377,RUBIES,PRISM,6.6784,214.978311,52.953821,3.021473e+41,6.360257e+40,6.115780e+40,6.606596e+40,...,0.000000,6.423147e+28,1.283741e+44,6.785605e+28,1.356182e+44,7.172044e+28,1.433416e+44,3.744485e+27,7.483789e+42,18.121599
340,970128,RUBIES,GRATING,6.4858,214.880997,52.891214,1.100483e+42,8.597488e+40,8.403987e+40,8.793203e+40,...,0.000000,5.863490e+28,1.171887e+44,6.097533e+28,1.218663e+44,6.332991e+28,1.265722e+44,2.347507e+27,4.691766e+42,25.974503
341,975193,RUBIES,PRISM,6.9901,214.967248,52.963630,1.031260e+42,8.545624e+40,8.388811e+40,8.702756e+40,...,0.000000,3.024284e+28,6.044383e+43,3.212576e+28,6.420706e+43,3.388494e+28,6.772300e+43,1.821052e+27,3.639585e+42,17.641312


# Saving file

In [10]:
# renaming some columns for consistency
good_snr_sources.rename(columns={"Survey":"SURVEY", "MSA_ID":"MSA ID", "Z_SPEC":"Z SPEC", "ABSOLUTE MAG 1500 16% ERROR":"ABSOLUTE MAG 1500 84% ERROR", "ABSOLUTE MAG 1500 84% ERROR":"ABSOLUTE MAG 1500 16% ERROR"}, inplace=True)

# adjusting errors for plotting
good_snr_sources['MASS 16% ERROR'] = good_snr_sources['MASS MEDIAN'] - good_snr_sources['MASS 16% ERROR']
good_snr_sources['MASS 84% ERROR'] = good_snr_sources['MASS 84% ERROR'] - good_snr_sources['MASS MEDIAN']
good_snr_sources['UV LUMINOSITY DUST-CORRECTED 16% ERROR'] = good_snr_sources['UV LUMINOSITY DUST-CORRECTED MEDIAN'] - good_snr_sources['UV LUMINOSITY DUST-CORRECTED 16% ERROR']
good_snr_sources['UV LUMINOSITY DUST-CORRECTED 84% ERROR'] = good_snr_sources['UV LUMINOSITY DUST-CORRECTED 84% ERROR'] - good_snr_sources['UV LUMINOSITY DUST-CORRECTED MEDIAN']

good_snr_sources.to_csv('../outputs/checkpoints/sample_Balmer_UV_mass.csv',index=False)
good_snr_sources

,MSA ID,SURVEY,SPECTRA TYPE,Z SPEC,RA,DEC,H-ALPHA INTEGRATED LUMINOSITY,H-ALPHA INTEGRATED LUMINOSITY MEAN ERROR,H-ALPHA INTEGRATED LUMINOSITY 16% ERROR,H-ALPHA INTEGRATED LUMINOSITY 84% ERROR,...,A_V ERR,UV LUMINOSITY DENSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DUST-CORRECTED 16% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DUST-CORRECTED MEDIAN,UV LUMINOSITY DENSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DUST-CORRECTED 84% ERROR,UV LUMINOSITY DENSITY DUST-CORRECTED MEAN ERROR,UV LUMINOSITY DUST-CORRECTED MEAN ERROR,UV LUMINOSITY SNR
0,323,CEERS,GRATING,5.6670,215.125247,53.004816,2.890517e+42,5.500441e+41,6.499634e+41,5.147870e+41,...,0.000000,4.257835e+29,1.984466e+43,4.357127e+29,8.708226e+44,4.692123e+29,6.695268e+43,2.171436e+28,4.339867e+43,20.065652
1,355,CEERS,GRATING,6.1000,215.151954,52.992771,2.524020e+42,4.020752e+41,4.026490e+41,4.015207e+41,...,0.448831,1.395383e+30,8.802788e+43,1.439428e+30,2.876864e+45,1.499501e+30,1.200637e+44,5.205892e+28,1.040458e+44,27.649978
2,403,CEERS,GRATING,5.7610,214.885556,52.835043,4.193009e+42,7.291792e+41,7.269488e+41,7.314849e+41,...,0.500283,1.512146e+30,6.124479e+43,1.542789e+30,3.083444e+45,1.576406e+30,6.718764e+43,3.213034e+28,6.421622e+43,48.016593
3,603,CEERS,GRATING,6.0580,215.032515,52.897511,2.323581e+42,4.094067e+41,4.092556e+41,4.096007e+41,...,0.502659,9.257243e+28,6.890788e+42,9.602021e+28,1.919076e+44,1.000540e+29,8.061961e+42,3.740775e+27,7.476374e+42,25.668534
4,618,CEERS,GRATING,6.0510,215.126857,52.968083,1.447606e+42,6.203280e+41,6.227582e+41,6.181015e+41,...,1.262428,7.096244e+28,2.072064e+43,8.132993e+28,1.625473e+44,9.031340e+28,1.795452e+43,9.675484e+27,1.933758e+43,8.405774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,959801,RUBIES,PRISM AND GRATING,6.7292,214.800187,52.814506,7.300656e+41,7.639598e+40,7.686104e+40,7.594264e+40,...,0.000000,5.017103e+28,7.621844e+42,5.398459e+28,1.078945e+44,5.656307e+28,5.153387e+42,3.196019e+27,6.387616e+42,16.891200
339,966377,RUBIES,PRISM,6.6784,214.978311,52.953821,3.021473e+41,6.360257e+40,6.115780e+40,6.606596e+40,...,0.000000,6.423147e+28,7.244156e+42,6.785605e+28,1.356182e+44,7.172044e+28,7.723421e+42,3.744485e+27,7.483789e+42,18.121599
340,970128,RUBIES,GRATING,6.4858,214.880997,52.891214,1.100483e+42,8.597488e+40,8.403987e+40,8.793203e+40,...,0.000000,5.863490e+28,4.677622e+42,6.097533e+28,1.218663e+44,6.332991e+28,4.705910e+42,2.347507e+27,4.691766e+42,25.974503
341,975193,RUBIES,PRISM,6.9901,214.967248,52.963630,1.031260e+42,8.545624e+40,8.388811e+40,8.702756e+40,...,0.000000,3.024284e+28,3.763232e+42,3.212576e+28,6.420706e+43,3.388494e+28,3.515939e+42,1.821052e+27,3.639585e+42,17.641312


# Documenting range of UV absolute mag

In [11]:
print('absolute UV continuum magnitude range')
print('min value: ', min(good_snr_sources['ABSOLUTE MAG 1500 MEDIAN']))
print('max value: ', max(good_snr_sources['ABSOLUTE MAG 1500 MEDIAN']))
print('median value: ', np.median(good_snr_sources['ABSOLUTE MAG 1500 MEDIAN']))

absolute UV continuum magnitude range
min value:  -23.616570370089363
max value:  -15.527809485762
median value:  -19.550886214522


In [12]:
# ----------------------------------------------
z_bins = ['1-2.5', '2.5-4', '4-5.5', '5.5-7']
mass_bins = ['7-8.5', '8.5-11.5']
redshifts = [1, 2.5, 4, 5.5, 7.1]
masses = [6.5, 8.5, 11.8]
# ----------------------------------------------

good_snr_sources['Z BIN'] = pd.cut(good_snr_sources['Z SPEC'], bins=redshifts, labels=z_bins, right=False)
good_snr_sources['MASS BIN'] = pd.cut(good_snr_sources['MASS MEDIAN'], bins=masses, labels=mass_bins, right=False)

In [13]:
for i, z_bin in enumerate(z_bins):
    # current Z BIN
    df_z_bin = good_snr_sources[good_snr_sources['Z BIN'] == z_bin]
    
    for j, m_bin in enumerate(mass_bins):
        # current MASS BIN
        mass_bin_df = df_z_bin[df_z_bin['MASS BIN'] == m_bin]
        print()
        print('redshift bin:', z_bin)
        print('mass bin:', m_bin)
        print('absolute UV continuum magnitude range')
        print('min value: ', min(mass_bin_df['ABSOLUTE MAG 1500 MEDIAN']))
        print('max value: ', max(mass_bin_df['ABSOLUTE MAG 1500 MEDIAN']))
        print('median value: ', np.median(mass_bin_df['ABSOLUTE MAG 1500 MEDIAN']))


redshift bin: 1-2.5
mass bin: 7-8.5
absolute UV continuum magnitude range
min value:  -19.409366708678444
max value:  -15.772205720011137
median value:  -17.751397135547787

redshift bin: 1-2.5
mass bin: 8.5-11.5
absolute UV continuum magnitude range
min value:  -21.608273245186776
max value:  -15.527809485762
median value:  -19.094894708191013

redshift bin: 2.5-4
mass bin: 7-8.5
absolute UV continuum magnitude range
min value:  -20.37741670394651
max value:  -16.215401460751785
median value:  -18.57949184161704

redshift bin: 2.5-4
mass bin: 8.5-11.5
absolute UV continuum magnitude range
min value:  -22.35102235869577
max value:  -16.830670930363638
median value:  -19.93822858321074

redshift bin: 4-5.5
mass bin: 7-8.5
absolute UV continuum magnitude range
min value:  -20.606803302656687
max value:  -17.817766395751523
median value:  -19.3650242663816

redshift bin: 4-5.5
mass bin: 8.5-11.5
absolute UV continuum magnitude range
min value:  -22.192637765793144
max value:  -16.6876642